In [42]:
import traci
import pandas as pd
import random
import time
import pickle

sumo_cmd = ["/opt/ohpc/pub/apps/sumo/bin/sumo"]
sumo_config = ["-c", "./Sim/network-bus.sumocfg"]
sumo_cmd.extend(sumo_config)

In [43]:
edge_data_by_step = pickle.load(open('output/filtered_edge_data_by_step.pkl', 'rb'))

In [44]:
INTERVAL = 60

In [35]:
start_t = time.time()
traci.start(sumo_cmd, label=str(random.randint(10000, 50000)))

log = {}
while traci.simulation.getTime() < 32400:
    step = int(traci.simulation.getTime())
    if step % 200 == 0:
        print(step)

    if step % INTERVAL == 0:
        if step in edge_data_by_step:
            for item in edge_data_by_step[step]:
                traci.edge.setMaxSpeed(item['edge_id'], item['speed'])
    
    veh_ids = traci.vehicle.getIDList()
    for veh in veh_ids:
        if veh not in log:
            log[veh] = []
        edge = traci.vehicle.getRoadID(veh)
        log[veh].append({'step': step, 'speed': traci.vehicle.getSpeed(veh), 'edge': edge,                
            'distance': traci.vehicle.getDistance(veh), 'fuel_consumption': traci.vehicle.getFuelConsumption(veh),
            'position': traci.vehicle.getPosition(veh), 'waiting_time': traci.vehicle.getWaitingTime(veh),
            'accumulated_waiting_time': traci.vehicle.getAccumulatedWaitingTime(veh)})

    traci.simulationStep()

traci.close()

finish_t = time.time()

 Retrying in 1 seconds
Loading configuration ... done.
***Starting server on port 37569 ***


Loading net-file from './Sim/Chattanooga_SUMO_Network.net.xml' ... done (9609ms).


Loading additional-files from './Sim/type.xml' ... done (1ms).
Loading additional-files from './Sim/busStopsCARTA.add.xml' ... done (22ms).
Loading done.
Simulation version v1_10_0+1480-daaf0c12055 started with time: 23400.00
23400
23600
23800
24000
24200
24400
24600
24800
25000


25200
25400
25600
25800
26000
26200
26400
26600
26800
27000
27200
27400


27600
27800
28000


...

31400


31600


31800


32000


32200


In [36]:
finish_t - start_t

197.0361635684967

In [37]:
pickle.dump(log, open('output/bus_data_removed_background.pkl', 'wb'))